In [303]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [304]:
from collections import namedtuple
import utlis_2nd.cusdom as custom
import super_learn_task_expr2 as expr2
import warnings

read_abso_path="/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/complex_center_dataset/combined_data.pt"
warnings.filterwarnings("ignore")

#data description
Data_description=namedtuple("Data_description",["t_steps","vesting_s","vari_numb","sample_rate","freq_numb"])
#temp info
Soft_arg_temp = namedtuple("Soft_arg_temp", ["learnable", "value"])
Gumble_temp = namedtuple("Gumble_temp", ["learnable", "value"])
Sample_info = namedtuple("Sample_info", ["type", "sample_numb"])
#loss regularization
Sinkhorn_loss_info = namedtuple("Sinkhorn_loss_info", ["loss_en", "p","blur"])
Fourier_loss_info = namedtuple("Fourier_loss_info", ["loss_en","labmbda_fourier"])
Lasso_loss_info = namedtuple("Lasso_loss_info", ["loss_en", "labmbda_lasso"])

In [305]:
# split data & get the t_steps
train_loader,valid_loader,test_loader,yaml_config =\
    custom.return_train_valid_test4loader(abso_path=read_abso_path)
# look（shape）
for batch_idx, (data, target) in enumerate(train_loader):
    print("shape:", data.shape)
    print("label for csv:",target.shape)
    break


load data & read yaml
shape: torch.Size([256, 100, 9])
label for csv: torch.Size([256])


In [306]:
soft_arg_temp = Soft_arg_temp(
                                learnable=yaml_config["Soft_argmax_info"][0],
                                value=yaml_config["Soft_argmax_info"][1])
gumble_temp = Gumble_temp(
                            learnable=yaml_config["Gumble_info"][0],
                            value=yaml_config["Gumble_info"][1])
sample_info = Sample_info(
                            type=yaml_config["Sample_info"][0],
                            sample_numb=yaml_config["Sample_info"][1])

In [307]:
results_path= "/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/expr2/expr2_results/expr2_"
expr2.config["train_nomin"]= int((yaml_config['train_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr2.config["valid_nomin"]= int((yaml_config['valid_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr2.config["test_nomin"]= int((yaml_config['test_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr2.config["train_loader"]= train_loader
expr2.config["valid_loader"]= valid_loader
expr2.config["test_loader"]= test_loader
expr2.config["device"] = "cpu"
expr2.config["data_description"]=yaml_config["data_description"]
expr2.config["hidden_act"]= "rational"
expr2.config["SI_lr"]= 1e-5
expr2.config["inference_output_act"]="Identity"
expr2.config["vari_number"]= yaml_config["vari_number"]
expr2.config["sample_vesting"]=2 #2s
#about temp
expr2.config["soft_arg_info"] = soft_arg_temp
expr2.config["gumble_info"] = gumble_temp
expr2.config["sample_info"] = sample_info
#about regularization
expr2.config["Sinkhorn_loss_info"] = yaml_config["Sinkhorn_loss_info"]
expr2.config["Fourier_loss_info"] = yaml_config["Fourier_loss_info"]
expr2.config["Lasso_loss_info"] = yaml_config["Lasso_loss_info"]
#about train step and regularization
expr2.config["Inference_num_epoch"]=10


In [308]:
# return test' results
result,co_actor=expr2.do_expr(
                                results_save_path=results_path,
                                folder_num=97,
                                train_config=expr2.config,
                                model_type="inference_net")

the prior knowledge is {'basis_1': 'x**0', 'basis_2': 'sin', 'basis_3': 'cos'}
output_dim 202
output_act is Identity
hi_inference_net
test_only_inference
start train_inference
sink_info [False, 2, 0.05, 1]
loss566.0959894527617_epoch0epoch_time2.261988878250122
loss521.4644824470789_epoch1epoch_time2.1759018898010254
loss486.67405364234963_epoch2epoch_time2.6299588680267334
loss460.04655094845657_epoch3epoch_time2.211615800857544
loss436.5663855883638_epoch4epoch_time2.1424498558044434
loss415.5187382718098_epoch5epoch_time2.463642120361328
loss396.8796992878489_epoch6epoch_time2.290325880050659
loss380.5692213744886_epoch7epoch_time2.1810848712921143
loss366.6489177952034_epoch8epoch_time2.245353937149048
loss355.0199113471562_epoch9epoch_time2.199828863143921


{'eval_freq_loss': array(1.76851472),
 'eval_mse_loss': array(388.35278288),
 'eval_u_stat_data': array(0.68348651),
 'eval_sinkhorn_data': array(139.72878279),
 'eval_mae': array(16.09859812)}

{'eval_freq_loss': array(2.36476879),
 'eval_mse_loss': array(327.51186632),
 'eval_u_stat_data': array(0.75337809),
 'eval_sinkhorn_data': array(283.83510756),
 'eval_mae': array(15.13508246)}